## Import Required Packages

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from collections import Counter
import cv2
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets
from torchvision import models
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.utils import make_grid